# Tentativa de análise estatística inferencial de dados econômicos do Brasil

### Dados analizados:
- https://sidra.ibge.gov.br/tabela/1737
- https://sidra.ibge.gov.br/tabela/1419

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
'''
Fonte: IBGE - Índice Nacional de Preços ao Consumidor Amplo
Notas
1 - Período de coleta ajustado ao mês civil (do primeiro ao trigésimo dia de cada mês), para os meses anteriores a março de 1986.

Legenda
Símbolo	Significado
-	Zero absoluto, não resultante de um cálculo ou arredondamento.
Ex: Em determinado município não existem pessoas de 14 anos de idade sem instrução.
0	Zero resultante de um cálculo ou arredondamento.
Ex: A inflação do feijão em determinada Região Metropolitana foi 0.
Determinado município produziu 400 kg de sementes de girassol e os dados da tabela são expressos em toneladas.
X	Valor inibido para não identificar o informante.
Ex: Determinado município só possui uma empresa produtora de cimento, logo o valor de sua produção deve ser inibido.
..	Valor não se aplica.
Ex: Não se pode obter o total da produção agrícola em determinado município quando os produtos agrícolas são contabilizados com unidades de medida distintas.
...	Valor não disponível.
Ex: A produção de feijão em determinado município não foi pesquisada ou determinado município não existia no ano da pesquisa.
A a Z
(exceto X)	Significa uma faixa de valores. Varia em função da tabela (se for o caso).
Ex: O nível de precisão da produção estimada de combustíveis está na faixa A (95 a 100%)
'''
pd.read_csv('tabela1737.tsv', sep='\t')

,Mês,Variável,Unidade territorial,Valor,Un
0,dezembro 1979,IPCA - Número-índice (base: dezembro de 1993 =...,Brasil,"0,0000000076183",Número-índice
1,dezembro 1979,IPCA - Variação mensal,Brasil,...,NaN
2,dezembro 1979,IPCA - Variação acumulada em 3 meses,Brasil,...,NaN
3,dezembro 1979,IPCA - Variação acumulada em 6 meses,Brasil,...,NaN
4,dezembro 1979,IPCA - Variação acumulada no ano,Brasil,...,NaN
...,...,...,...,...,...
2995,julho 2021,IPCA - Variação mensal,Brasil,"0,96",%
2996,julho 2021,IPCA - Variação acumulada em 3 meses,Brasil,"2,34",%
2997,julho 2021,IPCA - Variação acumulada em 6 meses,Brasil,"4,50",%
2998,julho 2021,IPCA - Variação acumulada no ano,Brasil,"4,76",%
